<a href="https://colab.research.google.com/github/sabeesh90/Layer_Fine_Tuning_Deep_Neural_Networks/blob/main/The_Lazarus_Trials_EfficientNetB2_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Importing libraries

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import seaborn as sns
import warnings
import os
from skimage import io
import pathlib
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from sklearn.metrics import classification_report, confusion_matrix
!pip install -q -U keras-tuner
import kerastuner as kt
%load_ext tensorboard
warnings.filterwarnings("ignore")


     |████████████████████████████████| 96 kB 2.8 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


In [ ]:
pip install Ipython

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
classes = ['GALAXY', 'QSO', 'STAR']

In [ ]:
def generate_graph(model_training):
    plt.figure(figsize= (15,5))
    plt.subplot(1,2,1)
    plt.plot(model_training.history.history['accuracy'])
    plt.plot(model_training.history.history['val_accuracy'])
    plt.title('Training vs TestAccuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(('Training', 'Validation'), loc = 'best')
    plt.subplot(1,2,2)
    plt.plot(model_training.history.history['loss'])
    plt.plot(model_training.history.history['val_loss'])
    plt.title('Training loss vs Test Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(('Training', 'Validation'), loc = 'best')
    plt.show()

class Metrics:
    # from sklearn
    def __init__(self, model, val_ds):
        self.val_ds = val_ds
        self.model = model
        self.prediction_classes = np.array([])
        self.true_classes = np.array([])
        
    def calculate_scores(self):     
        print('calculating score.....')   
        
        for x, y in self.val_ds:
            self.prediction_classes = np.concatenate([self.prediction_classes,np.argmax(self.model.predict(x), axis = -1)])
            self.true_classes = np.concatenate([self.true_classes, np.argmax(y.numpy(), axis=-1)])
        
        class_report = classification_report(self.true_classes, self.prediction_classes)
        conf_matrix = confusion_matrix(self.true_classes, self.prediction_classes)
        print(conf_matrix)
        print('------------------------------------')
        print(class_report)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=5,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=10,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

#### EfficientNetB2

##### Data preparation

In [ ]:
# Preprocessing the data
dataset_path = 'Images'
img_width = 512
img_height = 512
channels = 3
num_classes = 3
# batch_size = 8
batch_size = 1

img_width_crop = 1024
img_height_crop = 1024

rescale = tf.keras.Sequential([layers.experimental.preprocessing.Rescaling(1./255)])


# augmenting the data
# cropless flip and rotate

data_aug_flip = tf.keras.Sequential([layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                                # layers.experimental.preprocessing.RandomRotation(0.2)])
])

data_aug_flip_rotate = tf.keras.Sequential([layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                                layers.experimental.preprocessing.RandomRotation(0.2)])


# crop flip and rotate
data_aug_crop_flip_rotate = tf.keras.Sequential([layers.experimental.preprocessing.CenterCrop(512,512),
                                                 layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                                                 layers.experimental.preprocessing.RandomRotation(0.2)])

data_aug_crop_flip = tf.keras.Sequential([layers.experimental.preprocessing.CenterCrop(512,512),
                                          layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical")])

normal_crop = tf.keras.Sequential([layers.experimental.preprocessing.CenterCrop(512,512)],)
batch_size =32 
autotune = tf.data.AUTOTUNE

# Preparing the normal dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(dataset_path, validation_split = 0.2,subset = 'training',seed = 123, image_size = (img_width, img_height), 
                                                                  batch_size = batch_size, label_mode = 'categorical',)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(dataset_path, validation_split = 0.2,subset = 'validation',seed = 123, image_size = (img_width, img_height),
                                                                batch_size = batch_size, label_mode = 'categorical')

train_ds_cropped = tf.keras.preprocessing.image_dataset_from_directory(dataset_path, validation_split = 0.2,subset = 'training',seed = 123, image_size = (img_width_crop, img_height_crop), 
                                                                  batch_size = batch_size, label_mode = 'categorical',)
val_ds_cropped = tf.keras.preprocessing.image_dataset_from_directory(dataset_path, validation_split = 0.2,subset = 'validation',seed = 123, image_size = (img_width_crop, img_height_crop),
                                                                batch_size = batch_size, label_mode = 'categorical')


# preparing the scalred dataset
train_ds_scaled = train_ds.map(lambda x,y: (rescale(x, training = True), y), num_parallel_calls=autotune)
val_ds_scaled = val_ds.map(lambda x,y: (rescale(x, training = False), y), num_parallel_calls=autotune)
train_ds.prefetch(buffer_size=autotune)
val_ds.prefetch(buffer_size=autotune)

train_ds_scaled_cropped = train_ds_scaled.map(lambda x,y: (rescale(x, training = True), y), num_parallel_calls=autotune)
val_ds_scaled_cropped = val_ds_scaled.map(lambda x,y: (rescale(x, training = False), y), num_parallel_calls=autotune)

# augment function 
def prepare_dataset(ds, flip = False, flip_rotate = False):
    if flip == True:
        ds = ds.map(lambda x, y : (data_aug_flip(x, training = True), y), num_parallel_calls = autotune)
    elif flip_rotate == True:
        ds = ds.map(lambda x, y : (data_aug_flip_rotate(x, training = True), y), num_parallel_calls = autotune)
    
    return ds.prefetch(buffer_size =autotune)

def prepare_dataset_crop(ds, crop_flip = False, crop_flip_rotate = False):
    if crop_flip_rotate == True:
        ds = ds.map(lambda x, y : (data_aug_crop_flip_rotate(x, training = True), y), num_parallel_calls = autotune)
    elif crop_flip == True:
        ds = ds.map(lambda x, y : (data_aug_crop_flip(x, training = True), y), num_parallel_calls = autotune)
    
    return ds.prefetch(buffer_size =autotune)
    
def prepare_dataset_normal_crop(ds, training = True):
    ds = ds.map(lambda x, y : (normal_crop(x, training = training), y), num_parallel_calls = autotune)
    return ds.prefetch(buffer_size =autotune)
# 25 batches of 32 each and 7 batches of 32 each



Found 1000 files belonging to 3 classes.
Using 800 files for training.
Found 1000 files belonging to 3 classes.
Using 200 files for validation.
Found 1000 files belonging to 3 classes.
Using 800 files for training.
Found 1000 files belonging to 3 classes.
Using 200 files for validation.


##### EfficentNetB2 model building

In [ ]:
from tensorflow.keras.applications import  EfficientNetB2
models = {'base_model'+str(num) : EfficientNetB2(include_top=False, weights='imagenet', 
                                           input_shape = (img_height, img_width, 3),classes = num_classes, pooling = 'avg') for num in range(0,14)}

31801344/31790344 [==============================] - 0s 0us/step


In [ ]:
models

{'base_model0': <keras.engine.functional.Functional at 0x7fa3629353d0>,
 'base_model1': <keras.engine.functional.Functional at 0x7fa361e3b610>,
 'base_model10': <keras.engine.functional.Functional at 0x7fa33fd1a2d0>,
 'base_model11': <keras.engine.functional.Functional at 0x7fa33f763ad0>,
 'base_model12': <keras.engine.functional.Functional at 0x7fa33f0db9d0>,
 'base_model13': <keras.engine.functional.Functional at 0x7fa33ea496d0>,
 'base_model2': <keras.engine.functional.Functional at 0x7fa36194b790>,
 'base_model3': <keras.engine.functional.Functional at 0x7fa360995490>,
 'base_model4': <keras.engine.functional.Functional at 0x7fa36034dad0>,
 'base_model5': <keras.engine.functional.Functional at 0x7fa351c8b7d0>,
 'base_model6': <keras.engine.functional.Functional at 0x7fa351610c50>,
 'base_model7': <keras.engine.functional.Functional at 0x7fa35104ea50>,
 'base_model8': <keras.engine.functional.Functional at 0x7fa350a36110>,
 'base_model9': <keras.engine.functional.Functional at 0x7fa

In [ ]:
layer_count  = len(models['base_model0'].layers)
print(f'There are a total of {layer_count} in the resnet layer')

There are a total of 340 in the resnet layer


In [ ]:
for model,value in models.items():
    print(model)

base_model0
base_model1
base_model2
base_model3
base_model4
base_model5
base_model6
base_model7
base_model8
base_model9
base_model10
base_model11
base_model12
base_model13


In [ ]:
num=0
for i, (model_name, model) in zip(range(340, -1,-3), models.items()):
    break_point  = i
    for layer in model.layers[0:break_point]:
        layer.trainable = False
    for layer in model.layers[break_point:]:
        layer.trainable = True
    input_layer = model.input
    output_layer = model.output
    x = Dense(num_classes, activation  = 'softmax')(output_layer)    
    models[model_name] = Model(inputs = input_layer, outputs = x)


In [ ]:
for model_name, model in models.items():
    count = 0
    trained_layers = ['a' if layer.trainable == True else None for layer in model.layers]
    print(f'The number of trainable layers in {model_name} is', trained_layers.count('a'), sep=':')

The number of trainable layers in base_model0 is:1
The number of trainable layers in base_model1 is:4
The number of trainable layers in base_model2 is:7
The number of trainable layers in base_model3 is:10
The number of trainable layers in base_model4 is:13
The number of trainable layers in base_model5 is:16
The number of trainable layers in base_model6 is:19
The number of trainable layers in base_model7 is:22
The number of trainable layers in base_model8 is:25
The number of trainable layers in base_model9 is:28
The number of trainable layers in base_model10 is:31
The number of trainable layers in base_model11 is:34
The number of trainable layers in base_model12 is:37
The number of trainable layers in base_model13 is:40


In [ ]:
# resnet_model.summary()
for model_name, model in models.items():
    model.compile(optimizer = optimizers.Adam(lr=1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
i=0
for model_name, model in models.items():
    trainableParams = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])
    nonTrainableParams = np.sum([np.prod(v.get_shape()) for v in model.non_trainable_weights])
    totalParams = trainableParams + nonTrainableParams
    
    print('base_model'+str(i),trainableParams,nonTrainableParams,totalParams)
    i+=1


base_model0 4227 7768569.0 7772796.0
base_model1 7043 7765753.0 7772796.0
base_model2 502659 7270137.0 7772796.0
base_model3 1246787 6526009.0 7772796.0
base_model4 1620699 6152097.0 7772796.0
base_model5 1624923 6147873.0 7772796.0
base_model6 1648155 6124641.0 7772796.0
base_model7 2831579 4941217.0 7772796.0
base_model8 2962671 4810125.0 7772796.0
base_model9 2962671 4810125.0 7772796.0
base_model10 2976399 4796397.0 7772796.0
base_model11 3238479 4534317.0 7772796.0
base_model12 3498479 4274317.0 7772796.0
base_model13 3629571 4143225.0 7772796.0


##### EfficientNetB2 model with normalization of  image pixels

In [ ]:
models['base_model0'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 88s 2s/step - loss: 1.6074 - accuracy: 0.4325 - val_loss: 1.0575 - val_accuracy: 0.4350
Epoch 2/60
25/25 [==============================] - 11s 372ms/step - loss: 1.5971 - accuracy: 0.4775 - val_loss: 1.0787 - val_accuracy: 0.4350
Epoch 3/60
25/25 [==============================] - 11s 372ms/step - loss: 1.5938 - accuracy: 0.5000 - val_loss: 1.0731 - val_accuracy: 0.4350
Epoch 4/60
25/25 [==============================] - 11s 367ms/step - loss: 1.5932 - accuracy: 0.4888 - val_loss: 1.0752 - val_accuracy: 0.4350
Epoch 5/60
25/25 [==============================] - 11s 371ms/step - loss: 1.5942 - accuracy: 0.4925 - val_loss: 1.0706 - val_accuracy: 0.4350
Epoch 6/60
25/25 [==============================] - 11s 370ms/step - loss: 1.5969 - accuracy: 0.4787 - val_loss: 1.0833 - val_accuracy: 0.4350
Epoch 7/60
25/25 [==============================] - 11s 370ms/step - loss: 1.5931 - accuracy: 0.4900 - val_loss: 1.0826 - val_accuracy: 0.4350
Ep

In [ ]:
models['base_model1'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 19s 432ms/step - loss: 1.6074 - accuracy: 0.3237 - val_loss: 1.1117 - val_accuracy: 0.4350
Epoch 2/60
25/25 [==============================] - 11s 370ms/step - loss: 1.5954 - accuracy: 0.3713 - val_loss: 1.1127 - val_accuracy: 0.4350
Epoch 3/60
25/25 [==============================] - 11s 371ms/step - loss: 1.6005 - accuracy: 0.3525 - val_loss: 1.1040 - val_accuracy: 0.4350
Epoch 4/60
25/25 [==============================] - 11s 369ms/step - loss: 1.6023 - accuracy: 0.4150 - val_loss: 1.1071 - val_accuracy: 0.4350
Epoch 5/60
25/25 [==============================] - 11s 371ms/step - loss: 1.5879 - accuracy: 0.4350 - val_loss: 1.1089 - val_accuracy: 0.4350
Epoch 6/60
25/25 [==============================] - 11s 372ms/step - loss: 1.5959 - accuracy: 0.4575 - val_loss: 1.1152 - val_accuracy: 0.4350
Epoch 7/60
25/25 [==============================] - 11s 372ms/step - loss: 1.5966 - accuracy: 0.4450 - val_loss: 1.1214 - val_accuracy: 0.4350

In [ ]:
models['base_model2'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 20s 433ms/step - loss: 1.5946 - accuracy: 0.4563 - val_loss: 1.0577 - val_accuracy: 0.4800
Epoch 2/60
25/25 [==============================] - 11s 371ms/step - loss: 1.6132 - accuracy: 0.4812 - val_loss: 1.0753 - val_accuracy: 0.4800
Epoch 3/60
25/25 [==============================] - 11s 373ms/step - loss: 1.6059 - accuracy: 0.3300 - val_loss: 1.1053 - val_accuracy: 0.4800
Epoch 4/60
25/25 [==============================] - 11s 373ms/step - loss: 1.6146 - accuracy: 0.4487 - val_loss: 1.0701 - val_accuracy: 0.4800
Epoch 5/60
25/25 [==============================] - 11s 370ms/step - loss: 1.6064 - accuracy: 0.3587 - val_loss: 1.0627 - val_accuracy: 0.4800
Epoch 6/60
25/25 [==============================] - 11s 371ms/step - loss: 1.5913 - accuracy: 0.4725 - val_loss: 1.0612 - val_accuracy: 0.4800
Epoch 7/60
25/25 [==============================] - 11s 372ms/step - loss: 1.6033 - accuracy: 0.4250 - val_loss: 1.0636 - val_accuracy: 0.4800

In [ ]:
models['base_model3'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 18s 437ms/step - loss: 1.6269 - accuracy: 0.4800 - val_loss: 1.1364 - val_accuracy: 0.4350
Epoch 2/60
25/25 [==============================] - 11s 375ms/step - loss: 1.6087 - accuracy: 0.3325 - val_loss: 1.1334 - val_accuracy: 0.4350
Epoch 3/60
25/25 [==============================] - 11s 375ms/step - loss: 1.5936 - accuracy: 0.4150 - val_loss: 1.1245 - val_accuracy: 0.4350
Epoch 4/60
25/25 [==============================] - 11s 377ms/step - loss: 1.6203 - accuracy: 0.3700 - val_loss: 1.0964 - val_accuracy: 0.4350
Epoch 5/60
25/25 [==============================] - 11s 375ms/step - loss: 1.6145 - accuracy: 0.3775 - val_loss: 1.1226 - val_accuracy: 0.4350
Epoch 6/60
25/25 [==============================] - 11s 374ms/step - loss: 1.6217 - accuracy: 0.2988 - val_loss: 1.0890 - val_accuracy: 0.4350
Epoch 7/60
25/25 [==============================] - 11s 375ms/step - loss: 1.5907 - accuracy: 0.4387 - val_loss: 1.0855 - val_accuracy: 0.4350

In [ ]:
models['base_model4'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 20s 441ms/step - loss: 1.6200 - accuracy: 0.3775 - val_loss: 1.1661 - val_accuracy: 0.0850
Epoch 2/60
25/25 [==============================] - 11s 377ms/step - loss: 1.6255 - accuracy: 0.3675 - val_loss: 1.1456 - val_accuracy: 0.0850
Epoch 3/60
25/25 [==============================] - 11s 378ms/step - loss: 1.6246 - accuracy: 0.2125 - val_loss: 1.1495 - val_accuracy: 0.0850
Epoch 4/60
25/25 [==============================] - 11s 378ms/step - loss: 1.6155 - accuracy: 0.4925 - val_loss: 1.1327 - val_accuracy: 0.0850
Epoch 5/60
25/25 [==============================] - 11s 378ms/step - loss: 1.6036 - accuracy: 0.4200 - val_loss: 1.1594 - val_accuracy: 0.0850
Epoch 6/60
25/25 [==============================] - 11s 379ms/step - loss: 1.5965 - accuracy: 0.4013 - val_loss: 1.1300 - val_accuracy: 0.0850
Epoch 7/60
25/25 [==============================] - 11s 380ms/step - loss: 1.6002 - accuracy: 0.3500 - val_loss: 1.1244 - val_accuracy: 0.0850

In [ ]:
models['base_model5'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 19s 447ms/step - loss: 1.6591 - accuracy: 0.3413 - val_loss: 1.0605 - val_accuracy: 0.4350
Epoch 2/60
25/25 [==============================] - 11s 383ms/step - loss: 1.6171 - accuracy: 0.3650 - val_loss: 1.0762 - val_accuracy: 0.4350
Epoch 3/60
25/25 [==============================] - 11s 384ms/step - loss: 1.5994 - accuracy: 0.4288 - val_loss: 1.0546 - val_accuracy: 0.4350
Epoch 4/60
25/25 [==============================] - 11s 381ms/step - loss: 1.6427 - accuracy: 0.2900 - val_loss: 1.0701 - val_accuracy: 0.4350
Epoch 5/60
25/25 [==============================] - 11s 384ms/step - loss: 1.6129 - accuracy: 0.4500 - val_loss: 1.0606 - val_accuracy: 0.4350
Epoch 6/60
25/25 [==============================] - 11s 383ms/step - loss: 1.5926 - accuracy: 0.3638 - val_loss: 1.0565 - val_accuracy: 0.4350
Epoch 7/60
25/25 [==============================] - 11s 382ms/step - loss: 1.6237 - accuracy: 0.3663 - val_loss: 1.0812 - val_accuracy: 0.4350

In [ ]:
models['base_model6'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 19s 452ms/step - loss: 1.6384 - accuracy: 0.3288 - val_loss: 1.2364 - val_accuracy: 0.0850
Epoch 2/60
25/25 [==============================] - 11s 386ms/step - loss: 1.6139 - accuracy: 0.3800 - val_loss: 1.2152 - val_accuracy: 0.0850
Epoch 3/60
25/25 [==============================] - 11s 388ms/step - loss: 1.6763 - accuracy: 0.2875 - val_loss: 1.1551 - val_accuracy: 0.0850
Epoch 4/60
25/25 [==============================] - 11s 388ms/step - loss: 1.6177 - accuracy: 0.3187 - val_loss: 1.1631 - val_accuracy: 0.0850
Epoch 5/60
25/25 [==============================] - 11s 387ms/step - loss: 1.6328 - accuracy: 0.4212 - val_loss: 1.1565 - val_accuracy: 0.0850
Epoch 6/60
25/25 [==============================] - 11s 387ms/step - loss: 1.6272 - accuracy: 0.2875 - val_loss: 1.1492 - val_accuracy: 0.0850
Epoch 7/60
25/25 [==============================] - 11s 387ms/step - loss: 1.6179 - accuracy: 0.4538 - val_loss: 1.1009 - val_accuracy: 0.4800

In [ ]:
models['base_model7'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 20s 456ms/step - loss: 1.6659 - accuracy: 0.3275 - val_loss: 1.1730 - val_accuracy: 0.0850
Epoch 2/60
25/25 [==============================] - 11s 392ms/step - loss: 1.6766 - accuracy: 0.2912 - val_loss: 1.1693 - val_accuracy: 0.0850
Epoch 3/60
25/25 [==============================] - 11s 394ms/step - loss: 1.5937 - accuracy: 0.4412 - val_loss: 1.1444 - val_accuracy: 0.0850
Epoch 4/60
25/25 [==============================] - 11s 393ms/step - loss: 1.6582 - accuracy: 0.3313 - val_loss: 1.1586 - val_accuracy: 0.0850
Epoch 5/60
25/25 [==============================] - 11s 395ms/step - loss: 1.5861 - accuracy: 0.3088 - val_loss: 1.1334 - val_accuracy: 0.0850
Epoch 6/60
25/25 [==============================] - 11s 392ms/step - loss: 1.6618 - accuracy: 0.2950 - val_loss: 1.1005 - val_accuracy: 0.4350
Epoch 7/60
25/25 [==============================] - 11s 395ms/step - loss: 1.6077 - accuracy: 0.3738 - val_loss: 1.0726 - val_accuracy: 0.4350

In [ ]:
models['base_model8'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 19s 459ms/step - loss: 1.6467 - accuracy: 0.3375 - val_loss: 1.0959 - val_accuracy: 0.4800
Epoch 2/60
25/25 [==============================] - 11s 397ms/step - loss: 1.6372 - accuracy: 0.3438 - val_loss: 1.1191 - val_accuracy: 0.0850
Epoch 3/60
25/25 [==============================] - 11s 393ms/step - loss: 1.6650 - accuracy: 0.3487 - val_loss: 1.1348 - val_accuracy: 0.0850
Epoch 4/60
25/25 [==============================] - 11s 393ms/step - loss: 1.6180 - accuracy: 0.3487 - val_loss: 1.1343 - val_accuracy: 0.0850
Epoch 5/60
25/25 [==============================] - 11s 394ms/step - loss: 1.6315 - accuracy: 0.2637 - val_loss: 1.1008 - val_accuracy: 0.4800
Epoch 6/60
25/25 [==============================] - 11s 398ms/step - loss: 1.6011 - accuracy: 0.4137 - val_loss: 1.0797 - val_accuracy: 0.4800
Epoch 7/60
25/25 [==============================] - 11s 397ms/step - loss: 1.6256 - accuracy: 0.2925 - val_loss: 1.0974 - val_accuracy: 0.4800

In [ ]:
models['base_model9'].fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 19s 458ms/step - loss: 1.6036 - accuracy: 0.3675 - val_loss: 1.1538 - val_accuracy: 0.4350
Epoch 2/60
25/25 [==============================] - 11s 397ms/step - loss: 1.6828 - accuracy: 0.2937 - val_loss: 1.0878 - val_accuracy: 0.4350
Epoch 3/60
25/25 [==============================] - 11s 398ms/step - loss: 1.6446 - accuracy: 0.3313 - val_loss: 1.0702 - val_accuracy: 0.4350
Epoch 4/60
25/25 [==============================] - 11s 395ms/step - loss: 1.6233 - accuracy: 0.3775 - val_loss: 1.0746 - val_accuracy: 0.4350
Epoch 5/60
25/25 [==============================] - 11s 397ms/step - loss: 1.6227 - accuracy: 0.2825 - val_loss: 1.0639 - val_accuracy: 0.4350
Epoch 6/60
25/25 [==============================] - 11s 396ms/step - loss: 1.6263 - accuracy: 0.3862 - val_loss: 1.0763 - val_accuracy: 0.4350
Epoch 7/60
25/25 [==============================] - 11s 398ms/step - loss: 1.6004 - accuracy: 0.3300 - val_loss: 1.0806 - val_accuracy: 0.4350

In [ ]:
base_effnet= EfficientNetB2(include_top=False, weights='imagenet',input_shape = (img_height, img_width, 3),classes = num_classes, pooling = 'avg')
for layer in base_effnet.layers:
    layer.trainable = True
input_layer = base_effnet.input
output_layer = base_effnet.output
x = Dense(num_classes, activation  = 'softmax')(output_layer)    
base_effnet_retrained = Model(inputs = input_layer, outputs = x)

trained_layers = ['a' if layer.trainable == True else None for layer in base_effnet_retrained.layers]
print(f'The number of trainable layers in base_effnet_retrained is', trained_layers.count('a'), sep=':')

The number of trainable layers in base_effnet_retrained is:341


In [ ]:
base_effnet_retrained.compile(optimizer = optimizers.Adam(lr=1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
base_effnet_retrained.fit(train_ds_scaled,validation_data=val_ds_scaled,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60


ResourceExhaustedError: ignored

In [ ]:
models['base_model0'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 18s 437ms/step - loss: 1.5870 - accuracy: 0.5312 - val_loss: 1.0429 - val_accuracy: 0.6600
Epoch 2/60
25/25 [==============================] - 11s 373ms/step - loss: 1.5461 - accuracy: 0.6650 - val_loss: 1.0265 - val_accuracy: 0.7550
Epoch 3/60
25/25 [==============================] - 11s 374ms/step - loss: 1.5336 - accuracy: 0.6825 - val_loss: 1.0052 - val_accuracy: 0.7450
Epoch 4/60
25/25 [==============================] - 11s 377ms/step - loss: 1.5046 - accuracy: 0.6825 - val_loss: 0.9930 - val_accuracy: 0.7400
Epoch 5/60
25/25 [==============================] - 11s 375ms/step - loss: 1.4782 - accuracy: 0.7150 - val_loss: 0.9604 - val_accuracy: 0.7400
Epoch 6/60
25/25 [==============================] - 11s 373ms/step - loss: 1.4418 - accuracy: 0.7450 - val_loss: 0.9591 - val_accuracy: 0.7650
Epoch 7/60
25/25 [==============================] - 11s 373ms/step - loss: 1.4315 - accuracy: 0.7212 - val_loss: 0.9301 - val_accuracy: 0.7550

In [ ]:
models['base_model1'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 20s 440ms/step - loss: 1.5875 - accuracy: 0.3225 - val_loss: 1.0706 - val_accuracy: 0.4750
Epoch 2/60
25/25 [==============================] - 11s 374ms/step - loss: 1.4943 - accuracy: 0.4850 - val_loss: 1.0493 - val_accuracy: 0.4900
Epoch 3/60
25/25 [==============================] - 11s 375ms/step - loss: 1.4227 - accuracy: 0.6162 - val_loss: 1.0317 - val_accuracy: 0.5100
Epoch 4/60
25/25 [==============================] - 11s 375ms/step - loss: 1.3783 - accuracy: 0.6750 - val_loss: 1.0097 - val_accuracy: 0.5250
Epoch 5/60
25/25 [==============================] - 11s 376ms/step - loss: 1.3300 - accuracy: 0.6963 - val_loss: 0.9883 - val_accuracy: 0.5400
Epoch 6/60
25/25 [==============================] - 11s 381ms/step - loss: 1.3022 - accuracy: 0.7200 - val_loss: 0.9600 - val_accuracy: 0.5800
Epoch 7/60
25/25 [==============================] - 11s 376ms/step - loss: 1.2658 - accuracy: 0.7113 - val_loss: 0.9262 - val_accuracy: 0.6100

In [ ]:
models['base_model2'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 18s 435ms/step - loss: 1.4168 - accuracy: 0.6675 - val_loss: 0.9652 - val_accuracy: 0.6100
Epoch 2/60
25/25 [==============================] - 11s 375ms/step - loss: 1.1293 - accuracy: 0.7600 - val_loss: 0.8762 - val_accuracy: 0.6300
Epoch 3/60
25/25 [==============================] - 11s 376ms/step - loss: 0.9898 - accuracy: 0.7862 - val_loss: 0.8319 - val_accuracy: 0.6150
Epoch 4/60
25/25 [==============================] - 13s 476ms/step - loss: 0.8825 - accuracy: 0.8175 - val_loss: 0.7609 - val_accuracy: 0.6700
Epoch 5/60
25/25 [==============================] - 11s 375ms/step - loss: 0.8157 - accuracy: 0.8350 - val_loss: 0.6633 - val_accuracy: 0.7150
Epoch 6/60
25/25 [==============================] - 11s 377ms/step - loss: 0.7554 - accuracy: 0.8288 - val_loss: 0.5977 - val_accuracy: 0.7500
Epoch 7/60
25/25 [==============================] - 11s 376ms/step - loss: 0.7161 - accuracy: 0.8575 - val_loss: 0.5495 - val_accuracy: 0.7600

In [ ]:
models['base_model3'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 19s 444ms/step - loss: 1.4084 - accuracy: 0.5813 - val_loss: 1.0419 - val_accuracy: 0.4200
Epoch 2/60
25/25 [==============================] - 11s 383ms/step - loss: 1.0558 - accuracy: 0.7700 - val_loss: 0.9378 - val_accuracy: 0.5800
Epoch 3/60
25/25 [==============================] - 11s 380ms/step - loss: 0.8841 - accuracy: 0.8062 - val_loss: 0.8406 - val_accuracy: 0.6150
Epoch 4/60
25/25 [==============================] - 11s 383ms/step - loss: 0.7880 - accuracy: 0.7812 - val_loss: 0.7620 - val_accuracy: 0.6300
Epoch 5/60
25/25 [==============================] - 11s 380ms/step - loss: 0.7360 - accuracy: 0.8275 - val_loss: 0.6603 - val_accuracy: 0.7400
Epoch 6/60
25/25 [==============================] - 11s 381ms/step - loss: 0.6399 - accuracy: 0.8487 - val_loss: 0.6088 - val_accuracy: 0.7550
Epoch 7/60
25/25 [==============================] - 11s 381ms/step - loss: 0.6115 - accuracy: 0.8763 - val_loss: 0.5520 - val_accuracy: 0.7750

In [ ]:
models['base_model4'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 19s 445ms/step - loss: 1.3734 - accuracy: 0.6812 - val_loss: 0.9968 - val_accuracy: 0.4500
Epoch 2/60
25/25 [==============================] - 11s 383ms/step - loss: 0.9712 - accuracy: 0.7075 - val_loss: 0.8709 - val_accuracy: 0.7350
Epoch 3/60
25/25 [==============================] - 11s 383ms/step - loss: 0.7504 - accuracy: 0.8163 - val_loss: 0.7855 - val_accuracy: 0.7550
Epoch 4/60
25/25 [==============================] - 11s 383ms/step - loss: 0.6837 - accuracy: 0.8400 - val_loss: 0.7469 - val_accuracy: 0.7750
Epoch 5/60
25/25 [==============================] - 11s 383ms/step - loss: 0.6743 - accuracy: 0.8788 - val_loss: 0.5904 - val_accuracy: 0.8700
Epoch 6/60
25/25 [==============================] - 11s 384ms/step - loss: 0.6181 - accuracy: 0.8225 - val_loss: 0.5817 - val_accuracy: 0.8300
Epoch 7/60
25/25 [==============================] - 11s 384ms/step - loss: 0.5695 - accuracy: 0.8788 - val_loss: 0.5297 - val_accuracy: 0.8450

In [ ]:
models['base_model5'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 19s 450ms/step - loss: 1.3807 - accuracy: 0.5975 - val_loss: 1.1287 - val_accuracy: 0.1600
Epoch 2/60
25/25 [==============================] - 11s 388ms/step - loss: 0.9919 - accuracy: 0.7650 - val_loss: 0.9847 - val_accuracy: 0.4800
Epoch 3/60
25/25 [==============================] - 11s 391ms/step - loss: 0.8042 - accuracy: 0.8125 - val_loss: 0.8346 - val_accuracy: 0.6200
Epoch 4/60
25/25 [==============================] - 11s 387ms/step - loss: 0.7513 - accuracy: 0.8275 - val_loss: 0.7776 - val_accuracy: 0.6600
Epoch 5/60
25/25 [==============================] - 11s 388ms/step - loss: 0.6874 - accuracy: 0.8425 - val_loss: 0.5878 - val_accuracy: 0.7900
Epoch 6/60
25/25 [==============================] - 11s 388ms/step - loss: 0.6206 - accuracy: 0.8400 - val_loss: 0.5863 - val_accuracy: 0.7600
Epoch 7/60
25/25 [==============================] - 11s 387ms/step - loss: 0.5540 - accuracy: 0.8825 - val_loss: 0.5766 - val_accuracy: 0.7700

In [ ]:
models['base_model6'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 22s 465ms/step - loss: 1.3644 - accuracy: 0.6137 - val_loss: 0.9287 - val_accuracy: 0.5550
Epoch 2/60
25/25 [==============================] - 11s 392ms/step - loss: 0.9788 - accuracy: 0.7837 - val_loss: 0.8414 - val_accuracy: 0.6900
Epoch 3/60
25/25 [==============================] - 11s 395ms/step - loss: 0.7781 - accuracy: 0.8500 - val_loss: 0.7244 - val_accuracy: 0.7750
Epoch 4/60
25/25 [==============================] - 11s 392ms/step - loss: 0.6721 - accuracy: 0.8788 - val_loss: 0.6026 - val_accuracy: 0.8400
Epoch 5/60
25/25 [==============================] - 11s 392ms/step - loss: 0.6135 - accuracy: 0.8750 - val_loss: 0.5481 - val_accuracy: 0.8600
Epoch 6/60
25/25 [==============================] - 11s 393ms/step - loss: 0.5854 - accuracy: 0.8687 - val_loss: 0.4612 - val_accuracy: 0.8900
Epoch 7/60
25/25 [==============================] - 11s 391ms/step - loss: 0.5506 - accuracy: 0.8800 - val_loss: 0.4329 - val_accuracy: 0.9050

In [ ]:
models['base_model7'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 20s 461ms/step - loss: 1.2633 - accuracy: 0.6712 - val_loss: 0.9295 - val_accuracy: 0.6400
Epoch 2/60
25/25 [==============================] - 11s 400ms/step - loss: 0.8109 - accuracy: 0.7887 - val_loss: 0.7534 - val_accuracy: 0.7550
Epoch 3/60
25/25 [==============================] - 11s 399ms/step - loss: 0.6336 - accuracy: 0.8750 - val_loss: 0.5629 - val_accuracy: 0.8650
Epoch 4/60
25/25 [==============================] - 11s 397ms/step - loss: 0.5064 - accuracy: 0.8838 - val_loss: 0.4933 - val_accuracy: 0.8800
Epoch 5/60
25/25 [==============================] - 11s 398ms/step - loss: 0.4713 - accuracy: 0.9100 - val_loss: 0.4023 - val_accuracy: 0.8900
Epoch 6/60
25/25 [==============================] - 11s 398ms/step - loss: 0.4624 - accuracy: 0.9013 - val_loss: 0.3535 - val_accuracy: 0.8850
Epoch 7/60
25/25 [==============================] - 11s 395ms/step - loss: 0.4116 - accuracy: 0.9100 - val_loss: 0.3079 - val_accuracy: 0.9050

In [ ]:
models['base_model8'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 19s 461ms/step - loss: 1.3375 - accuracy: 0.6687 - val_loss: 0.9068 - val_accuracy: 0.4800
Epoch 2/60
25/25 [==============================] - 11s 400ms/step - loss: 0.8093 - accuracy: 0.8200 - val_loss: 0.6954 - val_accuracy: 0.8550
Epoch 3/60
25/25 [==============================] - 11s 398ms/step - loss: 0.6870 - accuracy: 0.8425 - val_loss: 0.5911 - val_accuracy: 0.8650
Epoch 4/60
25/25 [==============================] - 11s 398ms/step - loss: 0.5775 - accuracy: 0.8712 - val_loss: 0.4776 - val_accuracy: 0.8850
Epoch 5/60
25/25 [==============================] - 11s 400ms/step - loss: 0.4984 - accuracy: 0.8850 - val_loss: 0.3665 - val_accuracy: 0.9100
Epoch 6/60
25/25 [==============================] - 11s 399ms/step - loss: 0.4649 - accuracy: 0.9125 - val_loss: 0.3477 - val_accuracy: 0.9050
Epoch 7/60
25/25 [==============================] - 11s 401ms/step - loss: 0.3730 - accuracy: 0.9112 - val_loss: 0.2974 - val_accuracy: 0.9000

In [ ]:
models['base_model9'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 20s 463ms/step - loss: 1.3160 - accuracy: 0.7013 - val_loss: 0.8912 - val_accuracy: 0.5900
Epoch 2/60
25/25 [==============================] - 11s 400ms/step - loss: 0.8372 - accuracy: 0.8213 - val_loss: 0.7300 - val_accuracy: 0.7200
Epoch 3/60
25/25 [==============================] - 11s 399ms/step - loss: 0.6565 - accuracy: 0.8438 - val_loss: 0.5828 - val_accuracy: 0.8500
Epoch 4/60
25/25 [==============================] - 11s 400ms/step - loss: 0.5218 - accuracy: 0.8875 - val_loss: 0.4734 - val_accuracy: 0.8700
Epoch 5/60
25/25 [==============================] - 11s 398ms/step - loss: 0.4564 - accuracy: 0.8825 - val_loss: 0.4937 - val_accuracy: 0.8200
Epoch 6/60
25/25 [==============================] - 11s 399ms/step - loss: 0.4581 - accuracy: 0.9025 - val_loss: 0.3753 - val_accuracy: 0.8800
Epoch 7/60
25/25 [==============================] - 11s 397ms/step - loss: 0.4319 - accuracy: 0.9000 - val_loss: 0.3257 - val_accuracy: 0.8750

In [ ]:
models['base_model10'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 108s 3s/step - loss: 1.2335 - accuracy: 0.6900 - val_loss: 0.9440 - val_accuracy: 0.5200
Epoch 2/60
25/25 [==============================] - 12s 411ms/step - loss: 0.7770 - accuracy: 0.8238 - val_loss: 0.7931 - val_accuracy: 0.7200
Epoch 3/60
25/25 [==============================] - 12s 410ms/step - loss: 0.6073 - accuracy: 0.8687 - val_loss: 0.6724 - val_accuracy: 0.8000
Epoch 4/60
25/25 [==============================] - 12s 411ms/step - loss: 0.4814 - accuracy: 0.9013 - val_loss: 0.5172 - val_accuracy: 0.8800
Epoch 5/60
25/25 [==============================] - 12s 410ms/step - loss: 0.4633 - accuracy: 0.8963 - val_loss: 0.4557 - val_accuracy: 0.8600
Epoch 6/60
25/25 [==============================] - 12s 410ms/step - loss: 0.4412 - accuracy: 0.8938 - val_loss: 0.4017 - val_accuracy: 0.8700
Epoch 7/60
25/25 [==============================] - 12s 412ms/step - loss: 0.3752 - accuracy: 0.9187 - val_loss: 0.3410 - val_accuracy: 0.8650
E

In [ ]:
models['base_model11'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 23s 493ms/step - loss: 1.1997 - accuracy: 0.7450 - val_loss: 0.9282 - val_accuracy: 0.5250
Epoch 2/60
25/25 [==============================] - 12s 413ms/step - loss: 0.6987 - accuracy: 0.8388 - val_loss: 0.8010 - val_accuracy: 0.8350
Epoch 3/60
25/25 [==============================] - 12s 414ms/step - loss: 0.5394 - accuracy: 0.8725 - val_loss: 0.6652 - val_accuracy: 0.8350
Epoch 4/60
25/25 [==============================] - 12s 414ms/step - loss: 0.4418 - accuracy: 0.8988 - val_loss: 0.5993 - val_accuracy: 0.8450
Epoch 5/60
25/25 [==============================] - 12s 414ms/step - loss: 0.3793 - accuracy: 0.9175 - val_loss: 0.4740 - val_accuracy: 0.8650
Epoch 6/60
25/25 [==============================] - 12s 415ms/step - loss: 0.3709 - accuracy: 0.9250 - val_loss: 0.3783 - val_accuracy: 0.8750
Epoch 7/60
25/25 [==============================] - 12s 412ms/step - loss: 0.3452 - accuracy: 0.9325 - val_loss: 0.2808 - val_accuracy: 0.9100

In [ ]:
models['base_model12'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 23s 498ms/step - loss: 1.1875 - accuracy: 0.7175 - val_loss: 0.9167 - val_accuracy: 0.6650
Epoch 2/60
25/25 [==============================] - 12s 414ms/step - loss: 0.7283 - accuracy: 0.8263 - val_loss: 0.7739 - val_accuracy: 0.7500
Epoch 3/60
25/25 [==============================] - 12s 419ms/step - loss: 0.5504 - accuracy: 0.8750 - val_loss: 0.6366 - val_accuracy: 0.8400
Epoch 4/60
25/25 [==============================] - 12s 415ms/step - loss: 0.4496 - accuracy: 0.9062 - val_loss: 0.4976 - val_accuracy: 0.8750
Epoch 5/60
25/25 [==============================] - 12s 416ms/step - loss: 0.3829 - accuracy: 0.9125 - val_loss: 0.3892 - val_accuracy: 0.8850
Epoch 6/60
25/25 [==============================] - 12s 417ms/step - loss: 0.3624 - accuracy: 0.9212 - val_loss: 0.3028 - val_accuracy: 0.9150
Epoch 7/60
25/25 [==============================] - 12s 416ms/step - loss: 0.3057 - accuracy: 0.9362 - val_loss: 0.2704 - val_accuracy: 0.9000

In [ ]:
models['base_model13'].fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60
25/25 [==============================] - 23s 497ms/step - loss: 1.1516 - accuracy: 0.7138 - val_loss: 0.8425 - val_accuracy: 0.6750
Epoch 2/60
25/25 [==============================] - 12s 418ms/step - loss: 0.7600 - accuracy: 0.8288 - val_loss: 0.7695 - val_accuracy: 0.6550
Epoch 3/60
25/25 [==============================] - 12s 420ms/step - loss: 0.5810 - accuracy: 0.8575 - val_loss: 0.6593 - val_accuracy: 0.7200
Epoch 4/60
25/25 [==============================] - 12s 418ms/step - loss: 0.4284 - accuracy: 0.9137 - val_loss: 0.5472 - val_accuracy: 0.8200
Epoch 5/60
25/25 [==============================] - 12s 419ms/step - loss: 0.3678 - accuracy: 0.9175 - val_loss: 0.4120 - val_accuracy: 0.8950
Epoch 6/60
25/25 [==============================] - 12s 422ms/step - loss: 0.3748 - accuracy: 0.9250 - val_loss: 0.3554 - val_accuracy: 0.8850
Epoch 7/60
25/25 [==============================] - 12s 417ms/step - loss: 0.3480 - accuracy: 0.9250 - val_loss: 0.3630 - val_accuracy: 0.8750

In [ ]:
from tensorflow.keras.applications import  EfficientNetB2
base_EfficientNetB2= EfficientNetB2(include_top=False, weights='imagenet',input_shape = (img_height, img_width, 3),classes = num_classes, pooling = 'avg')
for layer in base_EfficientNetB2.layers:
    layer.trainable = True
input_layer = base_EfficientNetB2.input
output_layer = base_EfficientNetB2.output
x = Dense(num_classes, activation  = 'softmax')(output_layer)    
base_EfficientNetB2_retrained = Model(inputs = input_layer, outputs = x)

trained_layers = ['a' if layer.trainable == True else None for layer in base_EfficientNetB2_retrained.layers]
print(f'The number of trainable layers in base_EfficientNetB2_retrained is', trained_layers.count('a'), sep=':')

The number of trainable layers in base_EfficientNetB2_retrained is:341


In [ ]:
base_EfficientNetB2_retrained.compile(optimizer = optimizers.Adam(lr=1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
base_EfficientNetB2_retrained.fit(train_ds,validation_data=val_ds,epochs = 60, verbose = 1, workers = 3, class_weight = {0:1.13, 1:8.03, 2:1}, callbacks = [reduce_lr])

Epoch 1/60


ResourceExhaustedError: ignored